## Load and plot DR/Templeton ephys data

In [2]:
#Import packages
import pandas as pd
import h5py
import numpy as np
import scipy as sp
import scipy.signal as sg
import scipy.stats as st
import xarray as xr
import os
import matplotlib.pyplot as plt
from matplotlib import patches
import ast
import glob

from DR_analysis_utils import Session, makePSTH, make_neuron_time_trials_tensor

%matplotlib notebook

In [3]:
#Set paths to experiment folders
main_path = [
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-07-26_14-09-36_620263\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-07-27_13-57-17_620263\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-08-02_15-40-19_620264\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-09-19_13-48-26_628801\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-09-26_12-48-09_636397\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-09-27_11-37-08_636397\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-12-05_13-08-02_644547\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-12-06_12-35-35_644547\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2023-01-17_11-39-17_646318\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2023-01-18_10-44-55_646318\processed",
    r"Y:\2023-02-27_08-14-30_649944\processed",
    r"Y:\2023-02-28_09-33-43_649944\processed",
    # DR mice
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220815\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220816\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220817\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230123\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230124\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230125\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230126\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230130\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230131\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230201\processed", 
    r"\\allen\programs\mindscope\workgroups\np-exp\PilotEphys\Task 2 pilot\DRpilot_644864_20230202\processed",
    r"\\allen\programs\mindscope\workgroups\np-exp\PilotEphys\Task 2 pilot\DRpilot_644866_20230207\processed", 
    r"Y:\DRpilot_644866_20230208\processed",
    r"Y:\DRpilot_644866_20230209\processed",
    r"Y:\DRpilot_644866_20230210\processed",
    r"Y:\DRpilot_644867_20230220\processed",
    r"Y:\DRpilot_644867_20230221\processed",
    r"Y:\DRpilot_644867_20230222\processed",
    r"Y:\DRpilot_644867_20230223\processed",
    r"Y:\DRpilot_649943_20230213\processed", 
    r"Y:\DRpilot_649943_20230214\processed",
    r"Y:\DRpilot_649943_20230215\processed",
    r"Y:\DRpilot_649943_20230216\processed",
    
]

mouseIDs = ['620263','620263','620264','628801','636397','636397','644547','644547',
            '646318','646318','649944','649944',
           #DR mice
            '626791','626791','626791',
            '636766','636766','636766','636766',
            '644864','644864','644864','644864',
            '644866','644866','644866','644866',
            '644867','644867','644867','644867',
            '649943','649943','649943','649943',]
exp_nums = [1,2, 1, 1, 1,2, 1,2, 
            1,2, 1,2, 
           #DR mice
            1,2,3,
            1,2,3,4,
            1,2,3,4,
            1,2,3,4,
            1,2,3,4,
            1,2,3,4,]

### Load/initialize data from one session

In [4]:
sel_session=12
session=Session(path=main_path[sel_session]) 
session.assign_unit_areas()

probeA areas found
probeB areas found
probeC areas found
probeF areas found


In [ ]:
# session.assign_unit_areas()

In [ ]:
print(session.metadata['mouseID']+' session ',session.metadata['ephys_session_num'])

In [ ]:
# trials table
session.trials

In [5]:
# units table
session.units

,quality,cluster_id,probe,firing_rate,presence_ratio,isi_viol,amplitude_cutoff,isolation_distance,l_ratio,d_prime,...,spread,velocity_above,velocity_below,area,AP_coord,DV_coord,ML_coord,AP_coords,DV_coords,ML_coords
id,,,,,,,,,,,,,,,,,,,,,
2208151110000,mua,0,A,7.294557,0.99,0.075665,0.066695,13.735280,0.763557,2.726638,...,80.0,-0.081758,NaN,VPM,NaN,NaN,NaN,280.0,188.0,174.0
2208151110001,mua,1,A,9.935784,0.99,0.170615,0.199826,34.335362,0.245009,2.538814,...,80.0,0.000000,NaN,VPM,NaN,NaN,NaN,280.0,188.0,174.0
2208151110002,good,2,A,4.481252,0.99,0.053188,0.028146,9.644711,2.208028,2.251783,...,90.0,1.854271,NaN,VPM,NaN,NaN,NaN,280.0,188.0,173.0
2208151110003,mua,3,A,5.673959,0.99,0.311404,0.381861,13.877816,1.774743,2.103251,...,90.0,0.789782,NaN,VPM,NaN,NaN,NaN,280.0,188.0,174.0
2208151110004,good,4,A,19.871350,0.99,0.049522,0.022552,37.552305,0.128942,3.578176,...,90.0,0.732552,NaN,VPM,NaN,NaN,NaN,280.0,188.0,174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208151160576,mua,576,F,0.056994,0.23,0.000000,0.077802,28.441033,0.907986,5.273655,...,100.0,0.961474,-0.588658,MOs2/3,NaN,NaN,NaN,138.0,80.0,157.0
2208151160577,good,577,F,0.117934,0.93,0.000000,0.015442,27.773183,0.489880,6.384430,...,50.0,1.030151,-0.343384,PS,NaN,NaN,NaN,198.0,232.0,194.0
2208151160578,mua,578,F,0.068393,0.59,0.000000,0.117354,23.307601,0.875982,5.967396,...,60.0,0.137353,-0.343384,CP,NaN,NaN,NaN,169.0,137.0,175.0


In [ ]:
# spike times from one unit
session.spike_times[session.units.index[0]]

In [ ]:
# information about each frame
session.frames

In [ ]:
session.rf_trials

### Create a convenient data array using "xarray"

In [ ]:
# loop through sessions and make unit xarrays
time_before_flash = 0.5
trial_duration = 2
bin_size = 0.001


# Make tensor (3-D matrix [units,time,trials])
trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                              session.trials,time_before_flash, trial_duration, 
                                              bin_size)

# make xarray
session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                           coords={
                               "unit_id": session.good_units.index.values,
                               "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                               "trials": session.trials.index.values
                               })

### Plot unit density across probes

In [ ]:
probes = session.good_units['probe'].unique()
ch_bins=np.arange(0,384,8)

units_by_channel = session.good_units[['probe','peak_channel']].value_counts()

fig,ax=plt.subplots(1,len(probes))

for ip,pr in enumerate(probes):
    unit_hist,edges=np.histogram(units_by_channel[pr].sort_index().index,bins=ch_bins)
    ax[ip].plot(unit_hist,ch_bins[:-1])
    ax[ip].set_ylim([0,384])
    ax[ip].set_xlabel('n units')
    ax[ip].set_title('Probe'+pr)
    if ip>0:
        ax[ip].set_yticklabels([])
    else:
        ax[ip].set_ylabel('channel')
        

In [ ]:
#estimate fraction of units in cortex


#input channel rage and probe, caculate the fraction of units contained there
sel_probe='F'
channel_range=[150,350]

units_in_range=len(session.good_units.query('probe == @sel_probe and \
                                           peak_channel >= @channel_range[0] and \
                                           peak_channel < @channel_range[1]'))

probe_units=len(session.good_units.query('probe == @sel_probe'))

print('Probe'+sel_probe+' fraction units in channel range:')
print(units_in_range/probe_units)



### Plot rasters -- sorted by stimulus duration

In [ ]:
#Plot rasters: sort by stimulus duration

#select unit either by its location in the unit table or its ID    
unit_id=session.good_units.index[145]
# unit_id=10121

fig,ax=plt.subplots(2,2,figsize=(8,7))
ax=ax.flatten()
stim_types=['vis1','vis2','sound1','sound2']

color_options = ['green','blue','red','orange']

trial_color_map = {}
for ii,ss in enumerate(stim_types):
    trial_color_map[ss]=color_options[ii]


for si,ss in enumerate(stim_types):
    stim_trials = session.trials[session.trials['trialStimID']==ss]
    stim_trials_sorted = stim_trials.sort_values(by='trial_stim_dur',axis=0,ascending=True)
    
    sel_trials = session.trial_da.sel(trials=stim_trials_sorted.index.values)
    
    #find the ilocs of block transitions
    sel_trials_table = session.trials.loc[stim_trials.index.values]
    column_changes = sel_trials_table['trialstimRewarded'].shift() != sel_trials_table['trialstimRewarded']
    block_changes = sel_trials_table[column_changes]
    
    for it,tt in enumerate(sel_trials.trials.values):
        trial_spikes = sel_trials.sel(unit_id=unit_id,trials=tt)
        trial_color=[]

        trial_color = trial_color_map[session.trials['trialStimID'][tt]]

        trial_spike_times = trial_spikes.time[trial_spikes.values.astype('bool')]
        ax[si].vlines(trial_spike_times,ymin=it,ymax=it+1,linewidth=0.75,color='k')
        
        ax[si].vlines(stim_trials_sorted['trial_stim_dur'].loc[tt],ymin=it-.01,ymax=it+1.01,linewidth=1,color='k')

    if len(block_changes)>1:
        if block_changes.iloc[0]['trialstimRewarded']=='vis1':
            start_block=1
        elif block_changes.iloc[0]['trialstimRewarded']=='sound1':
            start_block=0

        for xx in np.asarray([0,2,4])+start_block:
            start_iloc=sel_trials_table.index.get_loc(block_changes.index[xx])
            if (xx+1)>(len(block_changes)-1):
                end_iloc=len(sel_trials_table)
            else:
                end_iloc=sel_trials_table.index.get_loc(block_changes.index[xx+1])
            temp_patch=patches.Rectangle([-0.5,start_iloc],1.5,end_iloc-start_iloc,
                                        color=[0.5,0.5,0.5],alpha=0.15)
            ax[si].add_patch(temp_patch)
        
    ax[si].set_xlim([-0.5,1.5])

    ax[si].axvline(0,color='k',linewidth=1)

    ax[si].set_title(ss)

fig.suptitle('unit:'+str(unit_id)+' Probe'+session.good_units['probe'].loc[unit_id]+
             ' ch:'+str(session.good_units['peak_channel'].loc[unit_id]))

fig.tight_layout()



### Plot heatmaps

In [ ]:
#Average & normalize responses of each unit to each stimulus
gwindow = sg.gaussian(15, std=5)

#find baseline mean and std per unit
baseline_mean_per_trial=session.trial_da.sel(time=slice(-0.5,-0.4)).mean(dim=["time"])
baseline_mean=baseline_mean_per_trial.mean(dim="trials").values
baseline_std=baseline_mean_per_trial.std(dim="trials").values

#find unique stimuli
stimuli = np.unique(session.trials['trialStimID'])

#remove catch trials for this plot
stimuli = stimuli[stimuli!='catch']

#pre-allocate array for average
unit_frs_by_stim = np.zeros((len(session.trial_da.unit_id),len(session.trial_da.time),len(stimuli)))

#normalize each unit's avg FRs to its baseline  FR
for ss,stim in enumerate(stimuli):
    stim_trials = (session.trials.query('trialStimID==@stim')).index.values
    unit_frs_by_stim[:,:,ss] = session.trial_da.sel(trials=stim_trials).mean(dim="trials").values
    
    # z-score each unit rel to its baseline
    unit_frs_by_stim[:,:,ss] = ((unit_frs_by_stim[:,:,ss].T- baseline_mean.T)/baseline_std.T).T
    
    for iu in range(0,len(session.trial_da.unit_id)):
        unit_frs_by_stim[iu,:,ss]=sg.convolve(unit_frs_by_stim[iu,:,ss],
                                                gwindow,mode='same')/np.sum(gwindow)

In [ ]:
#Find/define probe borders for plotting

probe_borders=np.where(session.good_units['probe'].iloc[:-1].values!=session.good_units['probe'].iloc[1:].values)[0]
all_edges=np.hstack([0,probe_borders,len(session.good_units)])
midpoints=all_edges[:-1]+(all_edges[1:]-all_edges[:-1])/2
probe_labels=session.good_units['probe'].iloc[midpoints.astype('int')].values

In [ ]:
# plot heatmaps
# ordered by ascending channel number - top is tip of probe

fig,ax=plt.subplots(1,4,figsize=(10,8))
for xx in range(0,len(stimuli)): 
    im = ax[xx].imshow(unit_frs_by_stim[:,:,xx],aspect='auto',vmin=-3,vmax=3,
                   cmap=plt.get_cmap('bwr'),interpolation='none',
                   extent=(-time_before_flash,trial_duration-time_before_flash,
                           0,unit_frs_by_stim.shape[0]))
    
    ax[xx].axvline(0,color='k',linestyle='--',linewidth=1)
    ax[xx].set_title(stimuli[xx])
    ax[xx].set_xlim(-0.5,1.5)
    ax[xx].hlines(unit_frs_by_stim.shape[0]-probe_borders,xmin=-0.5,xmax=1.5,
                   color='k',linewidth=1)
    ax[xx].set_yticks(unit_frs_by_stim.shape[0]-midpoints)
    ax[xx].set_yticklabels(probe_labels)
    if xx>0:
        ax[xx].set_yticklabels([])

# this adjusts the other plots to make space for the colorbar
fig.subplots_adjust(bottom=0.1, right=0.8, top=0.9, hspace=0.3) 
cax = plt.axes([0.85, 0.1, 0.025, 0.8])
cbar = fig.colorbar(im, cax=cax)
cbar.ax.set_ylabel('z-scored firing rates')


### Plot heatmaps by stimulus duration

In [ ]:
#Find/define probe borders for plotting

probe_borders=np.where(session.good_units['probe'].iloc[:-1].values!=session.good_units['probe'].iloc[1:].values)[0]
all_edges=np.hstack([0,probe_borders,len(session.good_units)])
midpoints=all_edges[:-1]+(all_edges[1:]-all_edges[:-1])/2
probe_labels=session.good_units['probe'].iloc[midpoints.astype('int')].values

In [ ]:
#Average & normalize responses of each unit to each stimulus

gwindow = sg.gaussian(15, std=5)

#find baseline mean and std per unit
baseline_mean_per_trial=session.trial_da.sel(time=slice(-0.5,-0.4)).mean(dim=["time"])
baseline_mean=baseline_mean_per_trial.mean(dim="trials").values
baseline_std=baseline_mean_per_trial.std(dim="trials").values

#find unique stimuli
stimuli = np.unique(session.trials['trialStimID'])

#remove catch trials for this plot
stimuli = stimuli[stimuli!='catch']

#find stimulus durations
stim_dur = session.trials['trial_stim_dur'].unique()
#remove duration = 0 for this plot
stim_dur = stim_dur[stim_dur!=0]

#pre-allocate array for average
unit_frs_by_stim = np.zeros((len(session.trial_da.unit_id),len(session.trial_da.time),len(stimuli),len(stim_dur)))

#loop through stimulus durations
for dd,dur in enumerate(stim_dur):
    #normalize each unit's avg FRs to its baseline  FR
    for ss,stim in enumerate(stimuli):
        
        stim_trials = (session.trials.query('trialStimID==@stim and trial_stim_dur==@dur')).index.values
        
#         baseline_mean_per_trial=session.trial_da.sel(time=slice(-0.5,-0.1),trials=stim_trials).mean(dim=["time"])
#         baseline_mean=baseline_mean_per_trial.mean(dim="trials").values
#         baseline_std=baseline_mean_per_trial.std(dim="trials").values
        
        unit_frs_by_stim[:,:,ss,dd] = session.trial_da.sel(trials=stim_trials).mean(dim="trials").values

        # z-score each unit rel to its baseline
        unit_frs_by_stim[:,:,ss,dd] = ((unit_frs_by_stim[:,:,ss,dd].T- baseline_mean.T)/baseline_std.T).T
        
        for iu in range(0,len(session.trial_da.unit_id)):
            unit_frs_by_stim[iu,:,ss,dd]=sg.convolve(unit_frs_by_stim[iu,:,ss,dd],
                                                     gwindow,mode='same')/np.sum(gwindow)
        

In [ ]:
# plot heatmaps
# ordered by ascending channel number - top is tip of probe

for dd,dur in enumerate(stim_dur):
    fig,ax=plt.subplots(1,4,figsize=(10,8))
    for xx in range(0,len(stimuli)): 
        im = ax[xx].imshow(unit_frs_by_stim[:,:,xx,dd],aspect='auto',vmin=-3,vmax=3,
                       cmap=plt.get_cmap('bwr'),interpolation='none',
                       extent=(-time_before_flash,trial_duration-time_before_flash,
                               0,unit_frs_by_stim.shape[0]))

        ax[xx].axvline(0,color='k',linestyle='--',linewidth=1)
        ax[xx].axvline(dur,color='k',linestyle='--',linewidth=1)
        ax[xx].set_title(stimuli[xx])
        ax[xx].set_xlim(-0.5,1.5)
        ax[xx].hlines(unit_frs_by_stim.shape[0]-probe_borders,xmin=-0.5,xmax=1.5,
                       color='k',linewidth=1)
        ax[xx].set_yticks(unit_frs_by_stim.shape[0]-midpoints)
        ax[xx].set_yticklabels(probe_labels)
        if xx>0:
            ax[xx].set_yticklabels([])
    
    fig.suptitle(str(dur)+' sec stimuli')
    # this adjusts the other plots to make space for the colorbar
    fig.subplots_adjust(bottom=0.1, right=0.8, top=0.9, hspace=0.3) 
    cax = plt.axes([0.85, 0.1, 0.025, 0.8])
    cbar = fig.colorbar(im, cax=cax)
    cbar.ax.set_ylabel('z-scored firing rates')


In [ ]:
# session.trial_da.sel(trials=stim_trials).mean(dim="trials").values.shape

### Plot PSTH according to probe and channel range

In [ ]:
# choose channel range and probe

channel_range=[0,50]
sel_probe='B'

stimuli = np.unique(session.trials['trialStimID'])

stim_PSTHs = {}

sel_units = session.good_units.query('probe == @sel_probe and \
                                           peak_channel >= @channel_range[0] and \
                                           peak_channel < @channel_range[1]')
for stim in stimuli:

    stim_trials = session.trials.query('trialStimID == @stim').index

    stim_PSTHs[stim]=[]
    stim_PSTHs[stim].append(session.trial_da.sel(
                            unit_id=sel_units.index,
                            trials=stim_trials).mean(dim=['trials']))


In [ ]:
# smooth each unit's PSTH
gwindow = sg.gaussian(50, std=25)
stim_PSTH_smooth={}
for stim in stimuli:
    stim_PSTH_smooth[stim]=np.zeros(stim_PSTHs[stim][0].shape)

    for iu,uu in enumerate(stim_PSTHs[stim][0].unit_id.values):
        stim_PSTH_smooth[stim][iu,:]=sg.convolve(stim_PSTHs[stim][0].sel(unit_id=uu),
                                                 gwindow,mode='same')/np.sum(gwindow)


In [ ]:
fig,ax=plt.subplots(1,1,figsize=(6,4))

for stim in stimuli:

    y=np.nanmean(stim_PSTH_smooth[stim],0)
    err=np.nanstd(stim_PSTH_smooth[stim],0)/np.sqrt(stim_PSTH_smooth[stim].shape[0])
    linex=ax.plot(stim_PSTHs[stim][0].time, y)
    plt.fill_between(stim_PSTHs[stim][0].time, y-err, y+err,
        alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())


ax.set_title('probe'+sel_probe+' ch'+str(channel_range[0])+'-'+
             str(channel_range[1])+' (n='+str(len(stim_PSTHs[stim][0].unit_id))+')')
ax.set_xlabel('time relative to stimulus onset (s)')
ax.set_ylabel('firing rate (Hz)')
ax.set_xlim([-0.4,1.4])

ax.set_yticks([0,5,10,15,20])
ax.legend(stimuli)

stim_lines=[0,0.5,0.75,1]
for sl in stim_lines:
    ax.axvline(sl,color='k',linestyle='--',linewidth=0.5)

fig.tight_layout()

### Bin by trials - do responses change over the course of the session?

In [ ]:
# choose channel range and probe

trial_bin_size = 40


channel_range=[0,100]
sel_probe='F'

stim = 'sound2'

stim_PSTHs = {}

sel_units = session.good_units.query('probe == @sel_probe and \
                                           peak_channel >= @channel_range[0] and \
                                           peak_channel < @channel_range[1]')

stim_trials = session.trials.query('trialStimID == @stim').index

trial_bins = np.arange(0,len(stim_trials),trial_bin_size)

mean_stim_PSTH = session.trial_da.sel(unit_id=sel_units.index,
                                      trials=stim_trials).mean(dim=['trials'])

trial_bin_PSTH = np.zeros((mean_stim_PSTH.shape[0],
                           mean_stim_PSTH.shape[1],
                           np.ceil(len(stim_trials)/trial_bin_size).astype(int)))

for it,tt in enumerate(trial_bins):

    trial_bin_PSTH[:,:,it]=session.trial_da.sel(unit_id=sel_units.index,
                                                trials=stim_trials[tt:tt+trial_bin_size]).mean(dim=['trials'])

# smooth each unit's PSTH
gwindow = sg.gaussian(50, std=25)
trial_bin_PSTH_smooth=np.zeros((trial_bin_PSTH.shape))

for it in range(0,trial_bin_PSTH.shape[2]):
    for iu in range(0,trial_bin_PSTH.shape[0]):
        trial_bin_PSTH_smooth[iu,:,it]=sg.convolve(trial_bin_PSTH[iu,:,it],gwindow,mode='same')/np.sum(gwindow)


In [ ]:
fig,ax=plt.subplots(1,1,figsize=(6,4))

for it in range(0,trial_bin_PSTH.shape[2]):
    
    y=np.nanmean(trial_bin_PSTH_smooth[:,:,it],0)
    err=np.nanstd(trial_bin_PSTH_smooth[:,:,it],0)/np.sqrt(trial_bin_PSTH_smooth[:,:,it].shape[0])
    linex=ax.plot(mean_stim_PSTH[0].time, y)
    plt.fill_between(mean_stim_PSTH[0].time, y-err, y+err,
        alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())


ax.set_title('probe'+sel_probe+' ch'+str(channel_range[0])+'-'+
             str(channel_range[1]))
ax.set_xlabel('time relative to stimulus onset (s)')
ax.set_ylabel('firing rate (Hz)')
ax.set_xlim([-0.4,1.4])

ax.set_yticks([0,5,10,15,20])
ax.legend(trial_bins)

stim_lines=[0,0.5,0.75,1]
for sl in stim_lines:
    ax.axvline(sl,color='k',linestyle='--',linewidth=0.5)

fig.tight_layout()

### area-specific plots

In [ ]:
# heatmaps

#Average & normalize responses of each unit to each stimulus
gwindow = sg.gaussian(15, std=5)

#find baseline mean and std per unit
baseline_mean_per_trial=session.trial_da.sel(time=slice(-0.5,-0.4)).mean(dim=["time"])
baseline_mean=baseline_mean_per_trial.mean(dim="trials").values
baseline_std=baseline_mean_per_trial.std(dim="trials").values

#find unique stimuli
stimuli = np.unique(session.trials['trialStimID'])

#remove catch trials for this plot
stimuli = stimuli[stimuli!='catch']

#pre-allocate array for average
unit_frs_by_stim = np.zeros((len(session.trial_da.unit_id),len(session.trial_da.time),len(stimuli)))

#normalize each unit's avg FRs to its baseline  FR
for ss,stim in enumerate(stimuli):
    if 'trialOptoVoltage' in session.trials:
        stim_trials = (session.trials.query('trialStimID==@stim and not trialOptoVoltage.isnull()')).index.values
    else:
        stim_trials = (session.trials.query('trialStimID==@stim')).index.values
    unit_frs_by_stim[:,:,ss] = session.trial_da.sel(trials=stim_trials).mean(dim="trials").values
    
    # z-score each unit rel to its baseline
    unit_frs_by_stim[:,:,ss] = ((unit_frs_by_stim[:,:,ss].T- baseline_mean.T)/baseline_std.T).T
    
    for iu in range(0,len(session.trial_da.unit_id)):
        unit_frs_by_stim[iu,:,ss]=sg.convolve(unit_frs_by_stim[iu,:,ss],
                                                gwindow,mode='same')/np.sum(gwindow)

In [ ]:
#Find/define probe borders for plotting

probe_borders=np.where(session.good_units['area'].iloc[:-1].values!=session.good_units['area'].iloc[1:].values)[0]
all_edges=np.hstack([0,probe_borders,len(session.good_units)])
midpoints=all_edges[:-1]+(all_edges[1:]-all_edges[:-1])/2
probe_labels=session.good_units['area'].iloc[midpoints.astype('int')].values

In [ ]:
session.good_units['area'].unique()

In [ ]:
# plot heatmaps
# ordered by ascending channel number - top is tip of probe

fig,ax=plt.subplots(1,4,figsize=(10,8))

for xx in range(0,len(stimuli)): 
    im = ax[xx].imshow(unit_frs_by_stim[:,:,xx],aspect='auto',vmin=-3,vmax=3,
                   cmap=plt.get_cmap('bwr'),interpolation='none',
                   extent=(-time_before_flash,trial_duration-time_before_flash,
                           0,unit_frs_by_stim.shape[0]))
    
    ax[xx].axvline(0,color='k',linestyle='--',linewidth=1)
    ax[xx].set_title(stimuli[xx])
    ax[xx].set_xlim(-0.5,1.5)
    ax[xx].hlines(unit_frs_by_stim.shape[0]-probe_borders,xmin=-0.5,xmax=1.5,
                   color='k',linewidth=1)
    ax[xx].set_yticks(unit_frs_by_stim.shape[0]-midpoints)
    ax[xx].set_yticklabels(probe_labels)
    if xx>0:
        ax[xx].set_yticklabels([])

fig.suptitle(session.metadata['mouseID']+' rec'+str(session.metadata['ephys_session_num']))
        
# this adjusts the other plots to make space for the colorbar
fig.subplots_adjust(bottom=0.1, right=0.8, top=0.9, hspace=0.3) 
cax = plt.axes([0.85, 0.1, 0.025, 0.8])
cbar = fig.colorbar(im, cax=cax)
cbar.ax.set_ylabel('z-scored firing rates')


In [ ]:
# plot vis vs. sound response rates
session.trials

In [ ]:
avg_run_speed=np.zeros(len(session.trials))

for tt in range(0,len(session.trials)):
    startFrame=session.trials['trialStartFrame'].iloc[tt]
    endFrame=session.trials['trialStimStartFrame'].iloc[tt]
    avg_run_speed[tt]=np.nanmean(session.frames['runningSpeed'][startFrame:endFrame])
    
session.trials['avg_run_speed'] = avg_run_speed

In [ ]:
# session.frames

In [ ]:
# plot smoothed response rate to each stimulus
stims=session.trials['trialStimID'].unique()

trials_to_smooth=5
gwindow = sg.gaussian(trials_to_smooth*3, std=trials_to_smooth)

fig,ax=plt.subplots(1,1)
for ss in stims:
    stimTrials=session.trials.query('trialStimID == @ss')
    ax.plot(stimTrials.index,sg.convolve(stimTrials['trial_response'].values,
                                         gwindow,mode='same')
                                        /np.sum(gwindow))
    
ax.legend(stims)
ax.set_title(mouseIDs[sel_session]+' rec'+str(exp_nums[sel_session]))

In [ ]:
# session.trials['trialStimID'].unique()

In [ ]:
# interpolate response rate to each stimulus across all trials

stims=session.trials['trialStimID'].unique()

stim_list=np.asarray(['vis1','vis2','sound1','sound2','catch'])
colors=np.asarray(['tab:blue','tab:green','tab:red','tab:orange','grey'])

trials_to_smooth=5
gwindow = sg.gaussian(trials_to_smooth*3, std=trials_to_smooth)

fig,ax=plt.subplots(1,1)

for ss in stims:
    plot_color=colors[stim_list==ss][0]
    
    stimTrials=session.trials.query('trialStimID == @ss')
    stimTrials[ss+'_smooth']=sg.convolve(stimTrials['trial_response'].values,gwindow,mode='same')/np.sum(gwindow)

    interp_func=sp.interpolate.interp1d(stimTrials.index,stimTrials[ss+'_smooth'])

    xnew=np.arange(np.min(stimTrials.index),np.max(stimTrials.index))
    temp_interp=interp_func(xnew)
    interp_full=np.zeros((len(session.trials)))
    interp_full[:]=np.nan
    interp_full[np.min(stimTrials.index):np.max(stimTrials.index)]=temp_interp
    
    session.trials[ss+'_interp']=interp_full
    
#     ax.plot(xnew,temp_interp)
    ax.plot(np.arange(0,len(session.trials)),interp_full,color=plot_color)

    
high_performance_trials=session.trials.query('abs(vis1_interp - sound1_interp)>=0.3').index
ax.plot(high_performance_trials,np.ones(len(high_performance_trials)),'k.')
ax.plot(sg.convolve(session.trials['avg_run_speed']
                    /session.trials['avg_run_speed'].max(),
                    gwindow,mode='same')/np.sum(gwindow),'k',linewidth=0.5)

ax.legend(stims)
ax.set_title(mouseIDs[sel_session]+' rec'+str(exp_nums[sel_session]))
ax.set_xlabel('trial number')
ax.set_ylabel('smoothed response rate to stimulus')

In [ ]:
# #calculate rolling dprime

# # cross-modal and intra-modal

# cross_modal_dprime = np.zeros(len(session.trials))
# cross_modal_dprime[:] = np.nan

# intra_modal_dprime = np.zeros(len(session.trials))
# intra_modal_dprime[:] = np.nan

# vis_block_trials = session.trials.query('trialstimRewarded == "vis1" and vis_autoreward_trials == False').index.values
# aud_block_trials = session.trials.query('trialstimRewarded == "sound1" and aud_autoreward_trials == False').index.values

# blocks = [vis_block_trials,aud_block_trials]
# block_ids= ['vis','aud']

# for bb,block in enumerate(blocks):
#     if block_ids[bb]=='vis':
#         target='vis1'
#         nontarget='sound1'
#     elif block_ids[bb]=='aud':
#         target='sound1'
#         nontarget='vis1'
        
    
        
#     cross_modal_dprime[block] = (st.norm.ppf(session.trials[target+'_interp'].loc[block]-.001) - 
#                                  st.norm.ppf(session.trials[nontarget+'_interp'].loc[block]+.001))
    
#     intra_modal_dprime[block] = np.nanmean([(st.norm.ppf(session.trials['vis1_interp'].loc[block]-.001) - 
#                                              st.norm.ppf(session.trials['vis2_interp'].loc[block]+.001)),
#                                             (st.norm.ppf(session.trials['sound1_interp'].loc[block]-.001) - 
#                                              st.norm.ppf(session.trials['sound2_interp'].loc[block]+.001))
#                                            ])



In [ ]:
#calculate rolling dprime

cross_modal_dprime = np.zeros(len(session.trials))
cross_modal_dprime[:] = np.nan
intra_modal_dprime = np.zeros(len(session.trials))
intra_modal_dprime[:] = np.nan

for tt,trial in session.trials.iterrows():
    if (trial['vis_autoreward_trials']==True)|(trial['aud_autoreward_trials']==True):
        continue
        
    if trial['trialstimRewarded'] == 'vis1':
        temp_hit=trial['vis1_interp']
        temp_fa=trial['sound1_interp']
        temp_intra_fa=trial['vis2_interp']
        
    elif trial['trialstimRewarded'] == 'sound1':
        temp_hit=trial['sound1_interp']
        temp_fa=trial['vis1_interp']
        temp_intra_fa=trial['sound2_interp']
        
        
    if temp_hit==1:
        temp_hit=0.999
    elif temp_hit==0:
        temp_hit=0.001      
    if temp_fa==1:
        temp_fa=0.999
    elif temp_fa==0:
        temp_fa=0.001
    if temp_intra_fa==1:
        temp_intra_fa=0.999
    elif temp_intra_fa==0:
        temp_intra_fa=0.001
   
    cross_modal_dprime[tt]=(st.norm.ppf(temp_hit) - st.norm.ppf(temp_fa))
    intra_modal_dprime[tt]=(st.norm.ppf(temp_hit) - st.norm.ppf(temp_intra_fa))
    
    
session.trials['cross_modal_dprime'] = cross_modal_dprime
session.trials['intra_modal_dprime'] = intra_modal_dprime

In [ ]:
# interpolate response rate to each stimulus across all trials

stims=session.trials['trialStimID'].unique()

stim_list=np.asarray(['vis1','vis2','sound1','sound2','catch'])
colors=np.asarray(['tab:blue','tab:green','tab:red','tab:orange','grey'])

fig,ax=plt.subplots(1,1)

ax.plot(np.arange(0,len(session.trials)),session.trials['cross_modal_dprime'],'k',linewidth=1.5)
ax.plot(np.arange(0,len(session.trials)),session.trials['intra_modal_dprime'],'k--',linewidth=1.5)       

ax.axhline(2,color='tab:blue',linewidth=1)

vis_autorewards=session.trials.query('vis_autoreward_trials == True').index.values
aud_autorewards=session.trials.query('aud_autoreward_trials == True').index.values

for xx in range(0,len(vis_autorewards)):
    ax.axvline(vis_autorewards[xx],color='b',alpha=0.5)
    ax.axvline(aud_autorewards[xx],color='r',alpha=0.5)
    if xx==0:
        ax.legend(['cross-modal dprime','intra-modal dprime','default threshold',
                  'vis autorewards','aud autorewards'])
        
ax.set_title(mouseIDs[sel_session]+' rec'+str(exp_nums[sel_session]))
ax.set_xlabel('trial number')
ax.set_ylabel('smoothed dprime')

In [ ]:
session.trials.columns

In [ ]:
colors[stim_list==ss][0]
# stim_list==ss

In [ ]:
session.trials.query('abs(vis1_interp - sound1_interp)>=0.4').index

In [ ]:
session.trials.query('trialOptoVoltage.isnull()')

In [ ]:
def average_across_trials(trials):
    ntrials_per_stim={}
    
    #Average & normalize responses of each unit to each stimulus
    gwindow = sg.gaussian(15, std=5)

    #find baseline mean and std per unit
    baseline_mean_per_trial=session.trial_da.sel(time=slice(-0.5,-0.4)).mean(dim=["time"])
    baseline_mean=baseline_mean_per_trial.mean(dim="trials").values
    baseline_std=baseline_mean_per_trial.std(dim="trials").values

    #find unique stimuli
    stimuli = np.unique(session.trials['trialStimID'])

    #remove catch trials for this plot
    stimuli = stimuli[stimuli!='catch']

    #pre-allocate array for average
    unit_frs_by_stim = np.zeros((len(session.trial_da.unit_id),len(session.trial_da.time),len(stimuli)))
    
    #normalize each unit's avg FRs to its baseline  FR
    for ss,stim in enumerate(stimuli):
        if 'trialOptoVoltage' in session.trials:
            stim_trials = (session.trials.query('trialStimID==@stim and trialOptoVoltage.isnull()')).index.values
        else:
            stim_trials = (session.trials.query('trialStimID==@stim')).index.values
        ntrials_per_stim[stim]=len(stim_trials)
        unit_frs_by_stim[:,:,ss] = session.trial_da.sel(trials=stim_trials).mean(dim="trials").values

        # z-score each unit rel to its baseline
        unit_frs_by_stim[:,:,ss] = ((unit_frs_by_stim[:,:,ss].T- baseline_mean.T)/baseline_std.T).T

        for iu in range(0,len(session.trial_da.unit_id)):
            unit_frs_by_stim[iu,:,ss]=sg.convolve(unit_frs_by_stim[iu,:,ss],
                                                    gwindow,mode='same')/np.sum(gwindow)
            
    return unit_frs_by_stim,ntrials_per_stim


def plot_heatmaps_with_borders(unit_frs_by_stim,trial_duration,time_before_flash,borders,
                               border_midpoints,border_labels,ntrials_per_stim,title):
    fig,ax=plt.subplots(1,4,figsize=(10,8))

    for xx in range(0,len(stimuli)): 
        im = ax[xx].imshow(unit_frs_by_stim[:,:,xx],aspect='auto',vmin=-3,vmax=3,
                       cmap=plt.get_cmap('bwr'),interpolation='none',
                       extent=(-time_before_flash,trial_duration-time_before_flash,
                               0,unit_frs_by_stim.shape[0]))

        ax[xx].axvline(0,color='k',linestyle='--',linewidth=1)
        ax[xx].set_title(stimuli[xx]+' n='+str(ntrials_per_stim[stimuli[xx]]))
        ax[xx].set_xlim(-0.5,1.5)
        ax[xx].hlines(unit_frs_by_stim.shape[0]-borders,xmin=-0.5,xmax=1.5,
                       color='k',linewidth=1)
        ax[xx].set_yticks(unit_frs_by_stim.shape[0]-border_midpoints)
        ax[xx].set_yticklabels(border_labels)
        if xx>0:
            ax[xx].set_yticklabels([])

    # this adjusts the other plots to make space for the colorbar
    fig.subplots_adjust(bottom=0.1, right=0.8, top=0.9, hspace=0.3) 
    cax = plt.axes([0.85, 0.1, 0.025, 0.8])
    cbar = fig.colorbar(im, cax=cax)
    cbar.ax.set_ylabel('z-scored firing rates')
    fig.suptitle(title)

In [ ]:
#Find/define area borders for plotting

#area borders
borders=np.where(session.good_units['area'].iloc[:-1].values!=session.good_units['area'].iloc[1:].values)[0]
all_edges=np.hstack([0,borders,len(session.good_units)])
border_midpoints=all_edges[:-1]+(all_edges[1:]-all_edges[:-1])/2
border_labels=session.good_units['area'].iloc[border_midpoints.astype('int')].values

# #probe borders
# borders=np.where(session.good_units['probe'].iloc[:-1].values!=session.good_units['probe'].iloc[1:].values)[0]
# all_edges=np.hstack([0,borders,len(session.good_units)])
# border_midpoints=all_edges[:-1]+(all_edges[1:]-all_edges[:-1])/2
# border_labels=session.good_units['probe'].iloc[border_midpoints.astype('int')].values

In [ ]:
#average across trials where mouse had good sound-performance & plot
attend_sound_trials=session.trials.query('(sound1_interp - vis1_interp)>=0.3')
title=mouseIDs[sel_session]+' rec'+str(exp_nums[sel_session])+' attend sound trials'

unit_frs_by_stim,ntrials_per_stim=average_across_trials(attend_sound_trials)

plot_heatmaps_with_borders(unit_frs_by_stim,trial_duration,time_before_flash,borders,
                               border_midpoints,border_labels,ntrials_per_stim,title)


In [ ]:
#average across trials where mouse had good vis-performance
attend_vis_trials=session.trials.query('(vis1_interp - sound1_interp)>=0.3')
title=mouseIDs[sel_session]+' rec'+str(exp_nums[sel_session])+' attend vis trials'

unit_frs_by_stim,ntrials_per_stim=average_across_trials(attend_vis_trials)

plot_heatmaps_with_borders(unit_frs_by_stim,trial_duration,time_before_flash,borders,
                               border_midpoints,border_labels,ntrials_per_stim,title)



In [ ]:
session.good_units['area'].unique()

In [ ]:
session.metadata['mouseID']+'_'+str(session.metadata['ephys_session_num'])

unit_summary={}
unit_summary[session.metadata['mouseID']+'_'+str(session.metadata['ephys_session_num'])]=session.good_units['area'].value_counts().T

pd.DataFrame(unit_summary).T

In [ ]:
session.good_units['area'].value_counts()

In [ ]:
# plot PSTHs comparing vis-attend and aud-attend:
attend_vis_trials=session.trials.query('(vis1_interp - sound1_interp)>=0.4')
attend_aud_trials=session.trials.query('(sound1_interp - vis1_interp)>=0.4')

# choose channel range and probe

sel_units_by='area'

#select units to plot by area
area_sel='AUD'
#or by channel/probe
channel_range=[120,350]
sel_probe='F'


if sel_units_by=='area':
    sel_units = session.good_units.query('area.str.contains(@area_sel)')
    fig_title=('area '+area_sel+' (n='+str(len(sel_units))+
               ')  vis_speed='+str(np.round(attend_vis_trials['avg_run_speed'].mean(),decimals=1))+
               '  aud_speed='+str(np.round(attend_aud_trials['avg_run_speed'].mean(),decimals=1)))
    
elif sel_units_by=='channels':
    sel_units = session.good_units.query('probe == @sel_probe and \
                                           peak_channel >= @channel_range[0] and \
                                           peak_channel < @channel_range[1]')
    fig_title=('probe'+sel_probe+' ch'+str(channel_range[0])+'-'+
               str(channel_range[1])+' (n='+str(len(sel_units))+')')
    
# average PSTH across selected units & each stimulus
stimuli = np.unique(session.trials['trialStimID'])
stim_PSTHs = {}

for stim in stimuli:
    stim_trials_attend_vis = attend_vis_trials.query('trialStimID == @stim').index
    stim_trials_attend_aud = attend_aud_trials.query('trialStimID == @stim').index
    
    stim_PSTHs[stim]={}
    stim_PSTHs[stim]['attend_vis']=[]
    stim_PSTHs[stim]['attend_aud']=[]
    
    stim_PSTHs[stim]['attend_vis_n']=len(stim_trials_attend_vis)
    stim_PSTHs[stim]['attend_aud_n']=len(stim_trials_attend_aud)
    
    stim_PSTHs[stim]['attend_vis'].append(session.trial_da.sel(
                                        unit_id=sel_units.index,
                                        trials=stim_trials_attend_vis).mean(dim=['trials']))
    
    stim_PSTHs[stim]['attend_aud'].append(session.trial_da.sel(
                                        unit_id=sel_units.index,
                                        trials=stim_trials_attend_aud).mean(dim=['trials']))

# smooth each unit's PSTH
gwindow = sg.gaussian(25, std=10)
stim_PSTH_smooth={}
for stim in stimuli:
    stim_PSTH_smooth[stim]={}
    stim_PSTH_smooth[stim]['attend_vis']=np.zeros(stim_PSTHs[stim]['attend_vis'][0].shape)
    stim_PSTH_smooth[stim]['attend_aud']=np.zeros(stim_PSTHs[stim]['attend_aud'][0].shape)
    for iu,uu in enumerate(stim_PSTHs[stim]['attend_vis'][0].unit_id.values):
        stim_PSTH_smooth[stim]['attend_vis'][iu,:]=sg.convolve(stim_PSTHs[stim]['attend_vis'][0].sel(unit_id=uu),
                                                                gwindow,mode='same')/np.sum(gwindow)
        stim_PSTH_smooth[stim]['attend_aud'][iu,:]=sg.convolve(stim_PSTHs[stim]['attend_aud'][0].sel(unit_id=uu),
                                                                gwindow,mode='same')/np.sum(gwindow)

In [ ]:
# stim_PSTHs[stim]['attend_vis']
np.round(attend_vis_trials['avg_run_speed'].mean(),decimals=1)
np.round(attend_aud_trials['avg_run_speed'].mean(),decimals=1)

In [ ]:
fig,ax=plt.subplots(5,1,figsize=(6,12),sharex=True,sharey=True)

trialconds=['attend_vis','attend_aud']

for ss,stim in enumerate(stimuli):
    ax[ss].set_title(stim+'; n_vis='+str(stim_PSTHs[stim]['attend_vis_n'])+
                     '; n_aud='+str(stim_PSTHs[stim]['attend_aud_n']))
    
    for tt,trial_type in enumerate(trialconds):
        y=np.nanmean(stim_PSTH_smooth[stim][trial_type],0)
        err=np.nanstd(stim_PSTH_smooth[stim][trial_type],0)/np.sqrt(stim_PSTH_smooth[stim][trial_type].shape[0])
        linex=ax[ss].plot(stim_PSTHs[stim][trial_type][0].time, y)
        ax[ss].fill_between(stim_PSTHs[stim][trial_type][0].time, y-err, y+err,
            alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())
        
        if (ss==0)&(tt==1):
            ax[ss].legend(trialconds)
        
        if tt==1:
            ax[ss].axvline(0,color='k',linestyle='--',linewidth=0.75)
            ax[ss].axvline(0.5,color='k',linestyle='--',linewidth=0.75)
            ax[ss].set_ylabel('FR (Hz)')
            
ax[ss].set_xlabel('time (s)')

fig.suptitle(fig_title)

# ax.set_xlabel('time relative to stimulus onset (s)')
# ax.set_ylabel('firing rate (Hz)')
# ax.set_xlim([-0.4,1.4])

# ax.legend(stimuli)

# stim_lines=[0,0.5]
# for sl in stim_lines:
#     ax.axvline(sl,color='k',linestyle='--',linewidth=0.5)

# fig.tight_layout()

In [ ]:
session.trials

In [ ]:
session.good_units['duration']

### separate trials by response type

In [ ]:
#### still need to filter by good-performance epochs!!!

sel_units_by='area'

#select units to plot by area
area_sel='MOs'

unit_type_sel='RS'

if unit_type_sel=='FS':
    sel_units = session.good_units.query('area.str.contains(@area_sel) and duration<=0.4')
elif unit_type_sel=='RS':
    sel_units = session.good_units.query('area.str.contains(@area_sel) and duration>0.4')
    
fig_title=('area '+area_sel+' '+unit_type_sel+' (n='+str(len(sel_units))+
           ')  vis_speed='+str(np.round(attend_vis_trials['avg_run_speed'].mean(),decimals=1))+
           '  aud_speed='+str(np.round(attend_aud_trials['avg_run_speed'].mean(),decimals=1)))

    
# average PSTH across selected units & each stimulus
stimuli = np.unique(session.trials['trialStimID'])
response_types = ['hit','miss','fa','cr']
stim_PSTHs = {}
stim_PSTHs_z = {}

# compute baseline mean and std
sel_baseline_trials=session.trials.query('cross_modal_dprime >= 1.5 and intra_modal_dprime >= 1.5').index
baseline = session.trial_da.sel(unit_id=sel_units.index,
                                 time=slice(-0.5,0),
                                 trials=sel_baseline_trials).mean(dim=['time'])
baseline_mean = baseline.mean(dim=['trials'])
baseline_std = baseline.std(dim=['trials'])

for stim in stimuli:
    stim_trials = session.trials.query('trialStimID == @stim and \
                                        cross_modal_dprime >= 1.5 and\
                                        intra_modal_dprime >= 1.5')
    
    stim_response_trials = {}
    stim_response_trials['hit'] = stim_trials.query('trial_response == True and trialStimID == trialstimRewarded')
    stim_response_trials['miss'] = stim_trials.query('trial_response == False and trialStimID == trialstimRewarded')
    stim_response_trials['fa'] = stim_trials.query('trial_response == True and trialStimID != trialstimRewarded')
    stim_response_trials['cr'] = stim_trials.query('trial_response == False and trialStimID != trialstimRewarded')
    stim_PSTHs[stim]={}
    stim_PSTHs_z[stim]={}
    
    for rr in response_types:
        stim_PSTHs[stim][rr+'_n']=[]
        stim_PSTHs[stim][rr+'_n']=len(stim_response_trials[rr])
        stim_PSTHs[stim][rr]=[]
        stim_PSTHs[stim][rr].append(session.trial_da.sel(
                                            unit_id=sel_units.index,
                                            trials=stim_response_trials[rr].index).mean(dim=['trials']))
        
        stim_PSTHs_z[stim][rr+'_n']=[]
        stim_PSTHs_z[stim][rr+'_n']=len(stim_response_trials[rr])
        stim_PSTHs_z[stim][rr]=[]
        stim_PSTHs_z[stim][rr].append((session.trial_da.sel(
                                      unit_id=sel_units.index,
                                      trials=stim_response_trials[rr].index).mean(dim=['trials'])-
                                      baseline_mean)/baseline_std)
        
        
        
##maybe try not smoothing??
# smooth each unit's PSTH
gwindow = sg.gaussian(20, std=5)
stim_PSTH_smooth={}
stim_PSTH_z_smooth={}
for stim in stimuli:
    stim_PSTH_smooth[stim]={}
    stim_PSTH_z_smooth[stim]={}
    
    for rr in response_types:
        stim_PSTH_smooth[stim][rr]=np.zeros(stim_PSTHs[stim][rr][0].shape)
        stim_PSTH_z_smooth[stim][rr]=np.zeros(stim_PSTHs[stim][rr][0].shape)
        for iu,uu in enumerate(stim_PSTHs[stim][rr][0].unit_id.values):
            stim_PSTH_smooth[stim][rr][iu,:]=sg.convolve(stim_PSTHs[stim][rr][0].sel(unit_id=uu),
                                                                    gwindow,mode='same')/np.sum(gwindow)
            stim_PSTH_z_smooth[stim][rr][iu,:]=sg.convolve(stim_PSTHs_z[stim][rr][0].sel(unit_id=uu),
                                                                    gwindow,mode='same')/np.sum(gwindow)



In [ ]:
#  plot PSTHs for each response type on separate plot for each stimulus

fig,ax=plt.subplots(5,1,figsize=(6,12),sharex=True,sharey=True)

# response_types = ['hit','miss','fa','cr']
# color_sel = ['tab:green','tab:blue','tab:red','k']

response_types = ['miss','cr','fa','hit']
color_sel = ['tab:blue','k','tab:red','tab:green']

for ss,stim in enumerate(stimuli):
    ax[ss].set_title(stim+'; n_hit='+str(stim_PSTHs[stim]['hit_n'])+
                     '; n_miss='+str(stim_PSTHs[stim]['miss_n'])+
                     '; n_fa='+str(stim_PSTHs[stim]['fa_n'])+
                     '; n_cr='+str(stim_PSTHs[stim]['cr_n']))
    line_all=[]
    for tt,trial_type in enumerate(response_types):
        y=np.nanmean(stim_PSTH_smooth[stim][trial_type],0)
        err=np.nanstd(stim_PSTH_smooth[stim][trial_type],0)/np.sqrt(stim_PSTH_smooth[stim][trial_type].shape[0])
        linex=ax[ss].plot(stim_PSTHs[stim][trial_type][0].time, y, color=color_sel[tt])
        line_all.append(linex[0])
        ax[ss].fill_between(stim_PSTHs[stim][trial_type][0].time, y-err, y+err,
            alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())
    
    if ss==0:      
        ax[ss].legend(line_all,response_types)
    ax[ss].axvline(0,color='k',linestyle='--',linewidth=0.75)
    ax[ss].axvline(0.5,color='k',linestyle='--',linewidth=0.75)
    ax[ss].set_ylabel('FR (Hz)')
    ax[ss].set_xlim([-0.25,0.75])
ax[ss].set_xlabel('time (s)')

fig.suptitle(fig_title)

# ax.set_xlabel('time relative to stimulus onset (s)')
# ax.set_ylabel('firing rate (Hz)')
# ax.set_xlim([-0.4,1.4])

# ax.legend(stimuli)

# stim_lines=[0,0.5]
# for sl in stim_lines:
#     ax.axvline(sl,color='k',linestyle='--',linewidth=0.5)

# fig.tight_layout()

In [ ]:
#  plot PSTHs for each response type on separate plot for each stimulus

fig,ax=plt.subplots(5,1,figsize=(6,12),sharex=True,sharey=True)

# response_types = ['hit','miss','fa','cr']
# color_sel = ['tab:green','tab:blue','tab:red','k']

response_types = ['miss','cr','fa','hit']
color_sel = ['tab:blue','k','tab:red','tab:green']

for ss,stim in enumerate(stimuli):
    ax[ss].set_title(stim+'; n_hit='+str(stim_PSTHs[stim]['hit_n'])+
                     '; n_miss='+str(stim_PSTHs[stim]['miss_n'])+
                     '; n_fa='+str(stim_PSTHs[stim]['fa_n'])+
                     '; n_cr='+str(stim_PSTHs[stim]['cr_n']))
    line_all=[]
    for tt,trial_type in enumerate(response_types):
        y=np.nanmean(stim_PSTH_z_smooth[stim][trial_type],0)
        err=np.nanstd(stim_PSTH_z_smooth[stim][trial_type],0)/np.sqrt(stim_PSTH_z_smooth[stim][trial_type].shape[0])
        linex=ax[ss].plot(stim_PSTHs_z[stim][trial_type][0].time, y, color=color_sel[tt])
        line_all.append(linex[0])
        ax[ss].fill_between(stim_PSTHs_z[stim][trial_type][0].time, y-err, y+err,
            alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())
    
    if ss==0:      
        ax[ss].legend(line_all,response_types)
    ax[ss].axvline(0,color='k',linestyle='--',linewidth=0.75)
    ax[ss].axvline(0.5,color='k',linestyle='--',linewidth=0.75)
    ax[ss].set_ylabel('z-scored FR')
    ax[ss].set_xlim([-0.25,0.75])
ax[ss].set_xlabel('time (s)')

fig.suptitle(fig_title)

# ax.set_xlabel('time relative to stimulus onset (s)')
# ax.set_ylabel('firing rate (Hz)')
# ax.set_xlim([-0.4,1.4])

# ax.legend(stimuli)

# stim_lines=[0,0.5]
# for sl in stim_lines:
#     ax.axvline(sl,color='k',linestyle='--',linewidth=0.5)

# fig.tight_layout()

In [ ]:
line_all

### look @ switching trials


In [ ]:
switch_to_vis_trials = session.trials.query('vis_autoreward_trials == True')

#specify which cue # each trial was
cue_number=np.tile([1,2,3,4,5],3)
switch_to_vis_trials['cue_number']=cue_number

#average across area units & cue number trials
#select units to plot by area
area_sel='AUD'


sel_units = session.good_units.query('area.str.contains(@area_sel)')
fig_title=('area '+area_sel+' (n='+str(len(sel_units))+
           ')  vis_speed='+str(np.round(attend_vis_trials['avg_run_speed'].mean(),decimals=1))+
           '  aud_speed='+str(np.round(attend_aud_trials['avg_run_speed'].mean(),decimals=1)))
    

# average PSTH across selected units & each stimulus
stimuli = np.unique(session.trials['trialStimID'])
stim_PSTHs = {}

for stim in stimuli:
    stim_PSTHs[stim]={}
    
    for cue_num in switch_to_vis_trials['cue_number'].unique():
        stim_trials = switch_to_vis_trials.query('trialStimID == @stim and \
                                                cue_number == @cue_num and \
                                                trial_response == True').index
        stim_PSTHs[stim][cue_num]=[]
        stim_PSTHs[stim][cue_num].append(session.trial_da.sel(
                                            unit_id=sel_units.index,
                                            trials=stim_trials).mean(dim=['trials']))

# smooth each unit's PSTH
gwindow = sg.gaussian(25, std=10)
stim_PSTH_smooth={}
for stim in stimuli:
    stim_PSTH_smooth[stim]={}
    for cue_num in switch_to_vis_trials['cue_number'].unique():
        stim_PSTH_smooth[stim][cue_num]=np.zeros(stim_PSTHs[stim][cue_num][0].shape)
        for iu,uu in enumerate(stim_PSTHs[stim][cue_num][0].unit_id.values):
            stim_PSTH_smooth[stim][cue_num][iu,:]=sg.convolve(stim_PSTHs[stim][cue_num][0].sel(unit_id=uu),
                                                                    gwindow,mode='same')/np.sum(gwindow)


In [ ]:
fig,ax=plt.subplots(2,1,figsize=(6,6),sharex=True,sharey=True)

trialconds=[1,2,3,4,5]

for ss,stim in enumerate(['vis1']):
    
    ax[ss].set_title(stim)
    
    for tt,trial_type in enumerate(trialconds):
        y=np.nanmean(stim_PSTH_smooth[stim][trial_type],0)
        err=np.nanstd(stim_PSTH_smooth[stim][trial_type],0)/np.sqrt(stim_PSTH_smooth[stim][trial_type].shape[0])
        linex=ax[ss].plot(stim_PSTHs[stim][trial_type][0].time, y)
        ax[ss].fill_between(stim_PSTHs[stim][trial_type][0].time, y-err, y+err,
            alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())
        
#         if (ss==0)&(tt==1):
#             ax[ss].legend(trialconds)
        
#         if tt==1:

    ax[ss].legend(trialconds)
    ax[ss].axvline(0,color='k',linestyle='--',linewidth=0.75)
    ax[ss].axvline(0.5,color='k',linestyle='--',linewidth=0.75)
    ax[ss].set_ylabel('FR (Hz)')
    ax[ss].set_xlim([-0.05,0.25])
            
ax[ss].set_xlabel('time (s)')

fig.suptitle(fig_title)

In [ ]:
session.trials.query('aud_autoreward_trials == True')

### loop through sessions, find number of units, trials, hit rate, false alarm rate

In [ ]:
#define whether trial stimuli were in the rewarded modality

def trialStimInModality(trials):

    trialStimInRewardedModality = np.zeros(len(trials)).astype(bool)
    trialRewardedModality = []
    trialStimModality = []

    for tt,trial in trials.iterrows():

        if trial['trialStimID']!='catch':
            rew_mod=trial['trialstimRewarded'][:-1]
            stim_mod=trial['trialStimID'][:-1]

            trialRewardedModality.append(rew_mod)
            trialStimModality.append(stim_mod)

            trialStimInRewardedModality[tt] = stim_mod==rew_mod
        else:
            trialRewardedModality.append('')
            trialStimModality.append('')

    trials['trialStimModality']=trialStimModality
    trials['trialRewardedModality']=trialRewardedModality   
    trials['trialStimInRewardedModality']=trialStimInRewardedModality
    
    return(trials)

In [ ]:
# loop through sessions

session_date=[]
mouse_id=[]

#units
total_n=[]
probeA_n=[]
probeB_n=[]
probeC_n=[]
probeD_n=[]
probeE_n=[]
probeF_n=[]

#trials
n_trials=[]
n_vis_block_trials=[]
n_aud_block_trials=[]
n_catch=[]

vis_go_500=[]
vis_go_750=[]
vis_go_1000=[]

vis_nogo_500=[]
vis_nogo_750=[]
vis_nogo_1000=[]

aud_go_500=[]
aud_go_750=[]
aud_go_1000=[]

aud_nogo_500=[]
aud_nogo_750=[]
aud_nogo_1000=[]

hit_rate=[]
fa_rate=[]
fa_rel_rate=[]
fa_irrel_rate=[]

for ss in main_path:
    session=Session(path=ss) 
    
#     # get mouse number and recording date
#     if "recordings" in ss:
#         sub1 = "recordings"
#     elif "Y:" in ss:
#         sub1 = "Y:"
#     sub2 = "\\processed"
#     idx1 = ss.index(sub1)
#     idx2 = ss.index(sub2)
#     exp_folder=ss[idx1 + len(sub1) + 1: idx2]
#     mouse=exp_folder[-6:]
#     exp=exp_folder[:-7]
    
    session_date.append(session.metadata['ephys_session_num'])
    mouse_id.append(session.metadata['mouseID'])
    
    session.trials = trialStimInModality(session.trials)
    
    #unit info table
    total_n.append(len(session.good_units))
    probeA_n.append(len(session.good_units.query('probe == "A"')))
    probeB_n.append(len(session.good_units.query('probe == "B"')))
    probeC_n.append(len(session.good_units.query('probe == "C"')))
    probeD_n.append(len(session.good_units.query('probe == "D"')))
    probeE_n.append(len(session.good_units.query('probe == "E"')))
    probeF_n.append(len(session.good_units.query('probe == "F"')))
    
    #trial info table
    n_trials.append(len(session.trials))
    n_vis_block_trials.append(len(session.trials.query('trialRewardedModality == "vis"')))
    n_aud_block_trials.append(len(session.trials.query('trialRewardedModality == "sound"')))
    
    vis_go_500.append(len(session.trials.query('trial_stim_dur == 0.5 and trialStimID == "vis1"')))
    vis_go_750.append(len(session.trials.query('trial_stim_dur == 0.75 and trialStimID == "vis1"')))
    vis_go_1000.append(len(session.trials.query('trial_stim_dur == 1.0 and trialStimID == "vis1"')))

    vis_nogo_500.append(len(session.trials.query('trial_stim_dur == 0.5 and trialStimID == "vis2"')))
    vis_nogo_750.append(len(session.trials.query('trial_stim_dur == 0.75 and trialStimID == "vis2"')))
    vis_nogo_1000.append(len(session.trials.query('trial_stim_dur == 1.0 and trialStimID == "vis2"')))

    aud_go_500.append(len(session.trials.query('trial_stim_dur == 0.5 and trialStimID == "sound1"')))
    aud_go_750.append(len(session.trials.query('trial_stim_dur == 0.75 and trialStimID == "sound1"')))
    aud_go_1000.append(len(session.trials.query('trial_stim_dur == 1.0 and trialStimID == "sound1"')))

    aud_nogo_500.append(len(session.trials.query('trial_stim_dur == 0.5 and trialStimID == "sound2"')))
    aud_nogo_750.append(len(session.trials.query('trial_stim_dur == 0.75 and trialStimID == "sound2"')))
    aud_nogo_1000.append(len(session.trials.query('trial_stim_dur == 1.0 and trialStimID == "sound2"')))
    
    n_catch.append(len(session.trials.query('trialStimID == "catch"')))
    
    hit_rate.append(session.trials.query('trialStimID == trialstimRewarded')['trial_response'].mean())
    fa_rate.append(session.trials.query('trialStimID != trialstimRewarded')['trial_response'].mean())
    
    fa_rel_rate.append(session.trials.query('trialStimID != trialstimRewarded and \
                                      trialStimInRewardedModality == True')['trial_response'].mean())
    fa_irrel_rate.append(session.trials.query('trialStimID != trialstimRewarded and \
                                      trialStimInRewardedModality == False')['trial_response'].mean())
    

In [ ]:
# if "Y:" in ss:
#     sub1 = "Y:"
# sub2 = "\\processed"
# idx1 = ss.index(sub1)
# idx2 = ss.index(sub2)
# exp_folder=ss[idx1 + len(sub1) + 1: idx2]
# mouse=exp_folder[-6:]
# exp=exp_folder[:-7]

# exp_folder

In [ ]:
# make dataframes

unit_summary={
    'session_date':session_date,
    'mouseID':mouse_id,
    'total_n':total_n,
    'probeA_n':probeA_n,
    'probeA_frac_ctx':np.zeros(len(total_n)),
    'probeB_n':probeB_n,
    'probeB_frac_ctx':np.zeros(len(total_n)),
    'probeC_n':probeC_n,
    'probeC_frac_ctx':np.zeros(len(total_n)),
    'probeD_n':probeD_n,
    'probeD_frac_ctx':np.zeros(len(total_n)),
    'probeE_n':probeE_n,
    'probeE_frac_ctx':np.zeros(len(total_n)),
    'probeF_n':probeF_n,
    'probeF_frac_ctx':np.zeros(len(total_n)),
}

unit_summary_df=pd.DataFrame.from_dict(unit_summary)

trial_summary={
    'session_date':session_date,
    'mouseID':mouse_id,
    'n_trials':n_trials,
    'n_vis_block_trials':n_vis_block_trials,
    'n_aud_block_trials':n_aud_block_trials,
    'n_catch':n_catch,

    'vis_go_500':vis_go_500,
    'vis_go_750':vis_go_750,
    'vis_go_1000':vis_go_1000,

    'vis_nogo_500':vis_nogo_500,
    'vis_nogo_750':vis_nogo_750,
    'vis_nogo_1000':vis_nogo_1000,

    'aud_go_500':aud_go_500,
    'aud_go_750':aud_go_750,
    'aud_go_1000':aud_go_1000,

    'aud_nogo_500':aud_nogo_500,
    'aud_nogo_750':aud_nogo_750,
    'aud_nogo_1000':aud_nogo_1000,

    'hit_rate':hit_rate,
    'fa_rate':fa_rate,
    'fa_rel_rate':fa_rel_rate,
    'fa_irrel_rate':fa_irrel_rate,
}

trial_summary_df=pd.DataFrame.from_dict(trial_summary)

In [ ]:
unit_summary_df[10:]

In [ ]:
pd.set_option('display.max_columns', None)
trial_summary_df.loc[10:,:]

In [ ]:
# # save dataframes
# save_path=r"C:\Users\ethan.mcbride\OneDrive - Allen Institute\templeton pilot summary"

# unit_summary_df.to_csv(os.path.join(save_path,'unit_summary.csv'))
# trial_summary_df.to_csv(os.path.join(save_path,'trial_summary.csv'))

### plot rasters - stim aligned w/ lick time indicated

In [ ]:
#pre - find first lick times in response window

#first - find trials where the mouse licked, find trials where mouse didn't lick

#next - sort first by stimulus and second by lick latency

In [ ]:
# find first licks after stimulus start

first_lick_time=np.zeros(len(session.trials))
first_lick_time[:]=np.nan

for tt in range(0,len(session.trials)):
    if tt<len(session.trials)-1:
        first_lick=np.where((session.lick_times>session.trials['stimStartTime'].iloc[tt])&
                            (session.lick_times<session.trials['stimStartTime'].iloc[tt+1]))[0]
    else:
        first_lick=np.where((session.lick_times>session.trials['stimStartTime'].iloc[tt]))[0]
    
    if len(first_lick)>0:
        first_lick_time[tt]=session.lick_times[first_lick[0]]
        
session.trials['first_lick_time']=first_lick_time
session.trials['first_lick_latency']=session.trials['first_lick_time']-session.trials['stimStartTime']

In [ ]:
response_trials = session.trials.query('trial_response == True')
non_response_trials = session.trials.query('trial_response == False')

In [ ]:
response_trials_sorted = response_trials.sort_values(by=['trialStimID','first_lick_latency'])
non_response_trials_sorted = non_response_trials.sort_values(by=['trialStimID'])

In [ ]:
#Plot rasters - resp vs. non-response, aligned to stimulus

#select unit either by its location in the unit table or its ID    
unit_id=session.good_units.index[310]
# unit_id=10121

fig,ax=plt.subplots(1,2,figsize=(10,7))
ax=ax.flatten()

# stim_types=['vis1','vis2','sound1','sound2','catch']
stim_types=['catch','sound2','sound1','vis2','vis1',]

for xx in range(0,2):
    ax[xx].axvline(0,linewidth=1)
    ax[xx].axvline(0.5,linewidth=1)
    
    trialcount_offset=0
    stim_trial_borders=[0]

    for si,ss in enumerate(stim_types):
        
        if xx==0:
            stim_trials = response_trials_sorted.query('trialStimID == @ss')
        elif xx==1:
            stim_trials = non_response_trials_sorted.query('trialStimID == @ss')
            
        sel_trials = session.trial_da.sel(trials=stim_trials.index.values)

        for it,tt in enumerate(sel_trials.trials.values):
            trial_spikes = sel_trials.sel(unit_id=unit_id,trials=tt)
            trial_spike_times = trial_spikes.time[trial_spikes.values.astype('bool')]
            ax[xx].vlines(trial_spike_times,ymin=it+trialcount_offset,ymax=it+1+trialcount_offset,linewidth=0.75,color='k')

            if stim_trials['trialStimID'].loc[tt] == stim_trials['trialstimRewarded'].loc[tt]:
                plot_color='g'
            else:
                plot_color='r'
            
            if xx==0:
                ax[xx].vlines(stim_trials['first_lick_latency'].loc[tt],ymin=it-.01+trialcount_offset,
                          ymax=it+1.01+trialcount_offset,linewidth=2,color=plot_color)
        ax[xx].axhline(trialcount_offset,color='k',linewidth=0.5)
        trialcount_offset=trialcount_offset+len(stim_trials)
        stim_trial_borders.append(trialcount_offset)

    stim_trial_borders=np.asarray(stim_trial_borders)
    stim_trial_midpoints=stim_trial_borders[:-1]+(stim_trial_borders[1:]-stim_trial_borders[:-1])/2

    for yy in np.asarray([1,3]):
        start_iloc=stim_trial_borders[yy]
        if (yy+1)>(len(stim_trial_borders)-1):
            end_iloc=stim_trial_borders[-1]
        else:
            end_iloc=stim_trial_borders[yy+1]
            
        temp_patch=patches.Rectangle([-0.5,start_iloc],1.5,end_iloc-start_iloc,
                                    color=[0.5,0.5,0.5],alpha=0.10)
        ax[xx].add_patch(temp_patch)

    
    ax[xx].axhline(trialcount_offset,color='k',linewidth=0.5)      
    ax[xx].set_xlim([-0.25,1])
    ax[xx].set_ylim([0,trialcount_offset])
    ax[xx].set_yticks(stim_trial_midpoints)
    ax[xx].set_yticklabels(stim_types)
    ax[xx].set_xlabel('time (s)')
    
    if xx==0:
        ax[xx].set_title('reponse trials')
    elif xx==1:
        ax[xx].set_title('non-reponse trials')
        
        
fig.suptitle('stim-aligned  unit:'+str(unit_id)+'  Probe'+session.good_units['probe'].loc[unit_id]+
             '  ch:'+str(session.good_units['peak_channel'].loc[unit_id])+'  area:'+
             session.good_units['area'].loc[unit_id])

fig.tight_layout()



### plot rasters - lick aligned w/ stim time indicated

In [ ]:
#make a dataframe with 'first_lick_time' relabeled as 'start_time' so the tensor function likes it
first_lick_df=response_trials['first_lick_time']
first_lick_df=pd.DataFrame(first_lick_df)
first_lick_df=first_lick_df.rename(columns={"first_lick_time": "start_time"})

In [ ]:
# loop through sessions and make unit xarrays
time_before_flash = 1.0
trial_duration = 3
bin_size = 0.001


# Make tensor (3-D matrix [units,time,trials])
trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                              first_lick_df, time_before_flash, trial_duration, 
                                              bin_size)

# make xarray
lick_aligned_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                           coords={
                               "unit_id": session.good_units.index.values,
                               "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                               "trials": first_lick_df.index.values
                               })

In [ ]:
#Plot rasters - resp vs. non-response, aligned to stimulus

#select unit either by its location in the unit table or its ID    
unit_id=session.good_units.index[12]
# unit_id=10121

fig,ax=plt.subplots(1,1,figsize=(10,7))

# stim_types=['vis1','vis2','sound1','sound2','catch']
stim_types=['catch','sound2','sound1','vis2','vis1',]


ax.axvline(0,linewidth=1)
# ax.axvline(0.5,linewidth=1)

trialcount_offset=0
stim_trial_borders=[0]

for si,ss in enumerate(stim_types):

    stim_trials = response_trials_sorted.query('trialStimID == @ss')

    sel_trials = lick_aligned_da.sel(trials=stim_trials.index.values)

    for it,tt in enumerate(sel_trials.trials.values):
        trial_spikes = sel_trials.sel(unit_id=unit_id,trials=tt)
        trial_spike_times = trial_spikes.time[trial_spikes.values.astype('bool')]
        ax.vlines(trial_spike_times,ymin=it+trialcount_offset,ymax=it+1+trialcount_offset,linewidth=0.75,color='k')

        if stim_trials['trialStimID'].loc[tt] == stim_trials['trialstimRewarded'].loc[tt]:
            plot_color='g'
        else:
            plot_color='r'


        ax.vlines(-stim_trials['first_lick_latency'].loc[tt],ymin=it-.01+trialcount_offset,
                  ymax=it+1.01+trialcount_offset,linewidth=2,color=plot_color)
        
    ax.axhline(trialcount_offset,color='k',linewidth=0.5)
    trialcount_offset=trialcount_offset+len(stim_trials)
    stim_trial_borders.append(trialcount_offset)

stim_trial_borders=np.asarray(stim_trial_borders)
stim_trial_midpoints=stim_trial_borders[:-1]+(stim_trial_borders[1:]-stim_trial_borders[:-1])/2

for yy in np.asarray([1,3]):
    start_iloc=stim_trial_borders[yy]
    if (yy+1)>(len(stim_trial_borders)-1):
        end_iloc=stim_trial_borders[-1]
    else:
        end_iloc=stim_trial_borders[yy+1]

    temp_patch=patches.Rectangle([-1,start_iloc],2,end_iloc-start_iloc,
                                color=[0.5,0.5,0.5],alpha=0.10)
    ax.add_patch(temp_patch)


ax.axhline(trialcount_offset,color='k',linewidth=0.5)      
ax.set_xlim([-1,1])
ax.set_ylim([0,trialcount_offset])
ax.set_yticks(stim_trial_midpoints)
ax.set_yticklabels(stim_types)
ax.set_xlabel('time (s)')

# if xx==0:
#     ax.set_title('reponse trials')
# elif xx==1:
#     ax.set_title('non-reponse trials')
        
        
fig.suptitle('lick-aligned  unit:'+str(unit_id)+'  Probe'+session.good_units['probe'].loc[unit_id]+
             '  ch:'+str(session.good_units['peak_channel'].loc[unit_id])+'  area:'+
             session.good_units['area'].loc[unit_id])

fig.tight_layout()

